In [1]:
import tensorflow as tf
from tensorflow.python.client import device_lib
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13706575592488834111
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 1433917851
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 2234059112605372495
 physical_device_desc: "device: 0, name: NVIDIA GeForce MX110, pci bus id: 0000:01:00.0, compute capability: 5.0"
 xla_global_id: 416903419]

In [3]:
#print("Runnning Jupyter Notebook using python version: {}".format(python_version()))
print("Running tensorflow version: {}".format(tf.keras.__version__))
print("Running tensorflow.keras version: {}".format(tf.__version__))
print("Running keras version: {}".format(tf.keras.__version__))
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.experimental.list_physical_devices('GPU')

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 2GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

Running tensorflow version: 2.7.0
Running tensorflow.keras version: 2.7.0
Running keras version: 2.7.0
Num GPUs Available:  1
1 Physical GPUs, 1 Logical GPUs


In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2

In [5]:
new_data = np.load('C://Users//hp//Desktop//Driver Drowsiness Detection System [Driver Alert]//Drowsiness Detection Approaches [ with Source Code ]//Approach B//Numpy Data//when-SF-1_044.npy', allow_pickle=True)

In [6]:
new_data

array([[array([[[  0, 255,   0],
                [  0, 255,   0],
                [  0, 255,   0],
                ...,
                [  0, 255,   0],
                [  0, 255,   0],
                [  0, 255,   0]],

               [[  0, 255,   0],
                [ 60,  66,  65],
                [ 34,  40,  39],
                ...,
                [100, 106, 105],
                [104, 110, 109],
                [ 42, 196,  44]],

               [[  0, 255,   0],
                [ 33,  39,  38],
                [ 17,  23,  22],
                ...,
                [109, 115, 114],
                [110, 116, 115],
                [ 44, 198,  46]],

               ...,

               [[  0, 255,   0],
                [ 96,  75,  54],
                [ 99,  76,  54],
                ...,
                [152, 130, 118],
                [103,  75,  60],
                [ 44, 183,  24]],

               [[  0, 255,   0],
                [ 97,  73,  55],
                [ 99,  75,  5

In [7]:
len(new_data)

2421

In [8]:
X = []
y = []
for feature, label in new_data:
    X.append(feature)
    y.append(label)

In [9]:
X = np.array(X)
X = X.reshape(-1, 145, 145, 3)

In [10]:
from sklearn.preprocessing import LabelBinarizer
label_bin = LabelBinarizer()
y = label_bin.fit_transform(y)

In [11]:
from sklearn.model_selection import train_test_split
seed = 42
test_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed, test_size=test_size)

In [12]:
print(len(X_test), len(y_test), "   ", len(X_train), len(y_train))

727 727     1694 1694


In [13]:
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import keras
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [14]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [15]:
train_generator = ImageDataGenerator(rescale=1/255, zoom_range=0.2, horizontal_flip=True, rotation_range=30)
test_generator = ImageDataGenerator(rescale=1/255)

train_generator = train_generator.flow(np.array(X_train), y_train, shuffle=False)
test_generator = test_generator.flow(np.array(X_test), y_test, shuffle=False)

In [16]:
model = Sequential()

model.add(Conv2D(256, (2, 2), activation="relu", input_shape=X_train.shape[1:]))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(128, (2, 2), activation="relu"))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(64, (2, 2), activation="relu"))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(32, (2, 2), activation="relu"))
model.add(MaxPooling2D(2, 2))

model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(64, activation="relu"))
model.add(Dense(4, activation="softmax"))

model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 144, 144, 256)     3328      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 72, 72, 256)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 71, 71, 128)       131200    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 35, 35, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 64)        32832     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 64)       0

In [ ]:
history = model.fit(train_generator, epochs=2, validation_data=test_generator, shuffle=True, batch_size=1)

Epoch 1/2
